Leiden followed by C means clustering of regulome

First make a proof of concept using igraph

Questions:
- How to measure distance
- How to measure membership
- How to treat small clusters - leave and expand or remove and add nodes to new?

If it works:
- make faster implementation

In [16]:
# Import packages

import igraph as ig
import pandas as pd
import numpy as np
import seaborn as sns
from compress_pickle import load, dump

def open_pickle(file):
    with open(path+file, 'rb') as pickle_file:
        return load(path=pickle_file, compression='infer')

path = './files/'

In [23]:
# load regulome
regulome_network_edges = pd.read_csv(filepath_or_buffer=path+'/human_regulome_pd.gz', compression='infer')
regulome_network_edges = regulome_network_edges.astype({'ProteinAid': 'str', 'ProteinBid':'str'})

regulome_graph = ig.Graph.DataFrame(regulome_network_edges, directed=False, use_vids=False)
msigdb_c3_tft_dict = open_pickle('msigdb_c3_tft_dict.pkl')

In [24]:
def leiden_clusters(graph, res, b, n_iter):
    return graph.community_leiden(objective_function='modularity',
                            weights='PPV',
                            resolution=res,
                            beta=b,
                            n_iterations=n_iter)

In [25]:
clusters = leiden_clusters(regulome_graph, 4, 0.1, 10)

RangeIndex(start=0, stop=501720, step=1)

In [ ]:
clusters_dict = {n:set(cluster) for (n, cluster) in enumerate(clusters)}
proteins = pd.concat([regulome_network_edges['ProteinAid'], regulome_network_edges['ProteinBid']]).unique()


In [ ]:
from itertools import chain

def network_c_means(graph, clusters):
    membership_dict = dict()

    # maybe remove small clusters before


    for node in graph.vs:
        node_membership = dict()
        node = node['name']
        neighbors = set(graph.neighbors(node))
        # maybe filter the clusters to which the node is conencted
        for (cluster_id, cluster) in clusters.items():
            connected_nodes = set(neighbors).intersection(cluster)
            
            if len(connected_nodes) == 0:
                node_membership[cluster_id] = 1
                continue

            weights = ([graph.es(graph.get_eid(node, connected_node))['PPV'] for connected_node in connected_nodes])
            weights_sum = sum(chain.from_iterable(weights))
            
            if weights_sum == 0: # dont need this myself, but a thing to look at if want to ship out later
                node_membership[cluster_id] = 1
                continue

            node_membership[cluster_id] = 1 / weights_sum
        
        membership_dict[node] = node_membership 
    return pd.DataFrame.from_dict(membership_dict, orient='columns')
            


network_c_means(regulome_graph, clusters_dict)

,349606,334435,346216,335411,351136,340227,338814,339117,347864,335179,...,351141,331039,351036,335118,335012,334209,339837,335013,336105,335283
0,0.016794,1.000000,0.150376,1.024590,1.000000,0.059844,1.000000,0.532481,1.000000,0.530786,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.176471
1,0.107980,1.083424,0.180930,1.000000,1.000000,0.132485,1.000000,1.000000,1.034126,1.046025,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
2,0.048707,1.000000,0.015162,1.000000,1.000000,0.041271,1.026694,0.353857,1.005025,0.537346,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
3,0.028873,1.000000,0.030075,0.360490,1.000000,0.026031,1.000000,1.170960,0.509424,0.363240,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
4,0.099731,1.054852,0.106157,1.140251,0.522739,0.056935,1.026694,1.040583,1.049318,0.550661,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.136631,1.000000,0.067531,1.000000,1.079914,0.077827,1.000000,1.119821,1.014199,1.106195,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
317,0.223614,1.000000,0.213858,1.176471,1.000000,0.181357,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000
318,0.525486,1.000000,0.269107,1.000000,1.000000,0.222173,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.176471,1.0,1.000000
319,0.361141,1.000000,0.184264,1.000000,1.000000,0.212404,1.000000,1.000000,1.000000,1.063830,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000


In [ ]:


def network_c_means(clusters, size_lim, fuzzy):
     

SyntaxError: invalid syntax (240424408.py, line 1)